In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import BCEWithLogitsLoss

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, TensorDataset
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [ ]:
# Define model name and number of labels
model_name = "sampathlonka/San-BERT"
num_labels = 21  # Replace with the actual number of labels in your data

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/472k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/951k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sampathlonka/San-BERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data = pd.read_excel('/content/modified_data.xlsx')

In [ ]:
test_split = 0.2

# Initial train and test split.
train_df, test_df = train_test_split(
    data,
    test_size=test_split,
)
print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 824
Number of rows in test set: 207


In [ ]:
# Split the data into texts and labels
train_texts = train_df['Text'].tolist()
train_labels = train_df.iloc[:, 1:].values.tolist()

In [ ]:
test_split = 0.5

# Initial train and test split.
val_df, test_df = train_test_split(
    test_df,
    test_size=test_split,
)
print(f"Number of rows in training set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 103
Number of rows in test set: 104


In [ ]:
# Split the data into texts and labels
val_texts = val_df['Text'].tolist()
val_labels = val_df.iloc[:, 1:].values.tolist()

In [ ]:
#train_texts
#train_labels

In [ ]:
# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)

In [ ]:
# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

In [ ]:
#train_encodings[0]
#train_encodings

In [ ]:
from torch.utils.data import Dataset
'''
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item'''

'\nclass CustomDataset(Dataset):\n    def __init__(self, encodings, labels):\n        self.encodings = encodings\n        self.labels = labels\n    \n    def __len__(self):\n        return len(self.encodings["input_ids"])\n    \n    def __getitem__(self, idx):\n        item = {key: val[idx] for key, val in self.encodings.items()}\n        item["labels"] = self.labels[idx]\n        return item'

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.encodings["input_ids"][idx]),
            'attention_mask': torch.tensor(self.encodings["attention_mask"][idx]),
            'labels': torch.tensor(self.labels[idx])
        }
        return item

In [ ]:
# Create custom dataset
train_dataset = CustomDataset(train_encodings, train_labels)

# Create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False)

In [ ]:
# Create custom dataset
val_dataset = CustomDataset(val_encodings, val_labels)

# Create dataloader
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [ ]:
#next(iter(train_dataset))
#next(iter(train_dataloader))

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    # Training loop
    model.train()  # Set the model to training mode
    train_loss = 0.0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/ {num_epochs}", leave=False):
        input_ids = batch['input_ids'].to(device)  # Move input to device
        attention_mask = batch['attention_mask'].to(device)  # Move attention mask to device
        labels = batch['labels'].to(device)  # Move labels to device

        optimizer.zero_grad()  # Clear gradients
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels.float())
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

        train_loss += loss.item()

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Validation", leave=False):
            input_ids = batch['input_ids'].to(device)  # Move input to device
            attention_mask = batch['attention_mask'].to(device)  # Move attention mask to device
            labels = batch['labels'].to(device)  # Move labels to device

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels.float())

            val_loss += loss.item()

    # Calculate average loss
    avg_train_loss = train_loss / len(train_dataloader)
    avg_val_loss = val_loss / len(val_dataloader)

    # Print epoch statistics
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")

Epoch 1/10, Train Loss: 0.2432224484323298, Val Loss: 0.21656023596341795


Epoch 2/10, Train Loss: 0.2429351686201628, Val Loss: 0.2170935393526004


Epoch 3/10, Train Loss: 0.2437858494740088, Val Loss: 0.2169903270327128


Epoch 4/10, Train Loss: 0.24276396067975795, Val Loss: 0.21655283581752044


Epoch 5/10, Train Loss: 0.24344961276476823, Val Loss: 0.21708473620506433


Epoch 6/10, Train Loss: 0.24247619404810147, Val Loss: 0.21688476319496447


Epoch 7/10, Train Loss: 0.24291172363225697, Val Loss: 0.21652013120742944


Epoch 8/10, Train Loss: 0.24196330566429397, Val Loss: 0.2173105741922672


Epoch 9/10, Train Loss: 0.24294886738061905, Val Loss: 0.2163731146317262


Epoch 10/10, Train Loss: 0.24292477466238355, Val Loss: 0.2173298385280829


In [ ]:
test_texts =test_df['Text'].tolist()
test_labels = test_df.iloc[:, 1:].values.tolist()
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)
test_dataset = CustomDataset(test_encodings, val_labels)
test_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Put the model in evaluation mode
model.eval()

# Lists to store predictions and labels
all_predictions = []
all_labels = []

# Iterate through the test dataset
for batch in tqdm(test_dataloader, desc="Testing", leave=False):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    # Disable gradient computation
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Apply sigmoid activation and round to obtain binary predictions
        predictions = torch.sigmoid(logits).cpu().numpy()
        rounded_predictions = (predictions > 0.5).astype(int)

        # Append predictions and labels to lists
        all_predictions.extend(rounded_predictions)
        all_labels.extend(labels.cpu().numpy())

# Convert lists to numpy arrays for easier manipulation
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

# Calculate evaluation metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')
conf_matrix = confusion_matrix(all_labels.argmax(axis=1), all_predictions.argmax(axis=1))
class_report = classification_report(all_labels, all_predictions)

# Print evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.0
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
Confusion Matrix:
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [23  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [14  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefi